In [1]:
import pathlib

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tifffile as tf  # write tiff files
from cytocherrypick.calculations import find_median
from PIL import Image  # read tiff files
from toml import load
from tqdm import tqdm  # progress bar

In [2]:
CELL_TYPE = "PBMC"

In [3]:
path = pathlib.Path(f"../results/{CELL_TYPE}/single_cell_predictions.parquet").resolve()

# read in the data
df = pd.read_parquet(path)
# show all columns
pd.set_option("display.max_columns", None)
df.head()

,oneb_Metadata_Treatment_Dose_Inhibitor_Dose,Metadata_Nuclei_Location_Center_Y,Metadata_Nuclei_Location_Center_X,Metadata_Image_Metadata_Site,group,Metadata_Site,Nuclei_RadialDistribution_ZernikePhase_CorrDNA_7_1,Cytoplasm_RadialDistribution_ZernikeMagnitude_CorrER_9_9,Cells_RadialDistribution_ZernikeMagnitude_CorrPM_6_4,Metadata_Well,Metadata_ImageNumber,Metadata_Cells_Number_Object_Number,comparison,treatment,feature,image_compsite_path,image_composite_crop_path,image_DAPI_path,image_ER_path,image_GasderminD_path,image_AGP_path,image_Mitochondria_path,image_compsite_crop_path,image_DAPI_crop_path,image_ER_crop_path,image_GasderminD_crop_path,image_AGP_crop_path,image_Mitochondria_crop_path
0,DMSO_0.100_%_DMSO_0.025_%,760.351675,1375.437799,9,Control,9,1.671473,-0.402880,0.834995,B07,89,1679,control_pyroptosis,control,Nuclei_RadialDistribution_ZernikePhase_CorrDNA...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...
1,DMSO_0.100_%_Z-VAD-FMK_100.000_uM,985.997753,242.062921,8,Control,8,1.671471,-0.128377,-0.758292,K12,344,1596,control_pyroptosis,control,Nuclei_RadialDistribution_ZernikePhase_CorrDNA...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...
2,H2O2_100.000_nM_DMSO_0.025_%,751.917676,1066.164649,12,Control,12,1.671471,-0.451012,-0.273077,H05,60,1437,control_pyroptosis,control,Nuclei_RadialDistribution_ZernikePhase_CorrDNA...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...
3,DMSO_0.100_%_Z-VAD-FMK_30.000_uM,2080.795367,131.328185,8,Control,8,-1.701544,0.038387,-0.643369,E12,344,2893,control_pyroptosis,control,Nuclei_RadialDistribution_ZernikePhase_CorrDNA...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home/lippincm/Documents/Interstellar_Analysis...,/home

In [4]:
df.columns
image_path_columns = [
    "image_compsite_path",
    "image_DAPI_path",
    "image_ER_path",
    "image_GasderminD_path",
    "image_AGP_path",
    "image_Mitochondria_path",
    "image_composite_crop_path",
    "image_DAPI_crop_path",
    "image_ER_crop_path",
    "image_GasderminD_crop_path",
    "image_AGP_crop_path",
    "image_Mitochondria_crop_path",
]
for path_type in image_path_columns:
    # print(path_type)
    for image_path in df[path_type]:
        # open the image
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        # image to array
        image_array = np.array(image)
        resolution = 3.3446  # pixels per micrometer
        # get the image size
        image_size = image_array.shape
        print(image_size)
        if image_array.shape[1] <= 500:
            scale_bar_length = 5  # um
            scale_bar_height = 1  # pixels
            padding = 3  # pixels
        elif image_array.shape[1] > 500:
            scale_bar_length = 100
            scale_bar_height = 10
            padding = 10
        # get the bottom right most corner based on scale 1 % of pixels
        scale_bar_x = (
            image_size[1] - (scale_bar_length * resolution) - padding - padding
        )
        scale_bar_y = image_size[0] - (scale_bar_height) - padding
        print(scale_bar_x, scale_bar_y)
        # draw the scale bar
        new = cv2.rectangle(
            image_array,
            (int(scale_bar_x), int(scale_bar_y)),
            (image_size[1] - padding, image_size[0] - padding),
            (255, 255, 255),
            -1,
        )
        # show the image
        # plt.imshow(new)
        # plt.show()
        # save the image with the scale bar via cv2
        cv2.imwrite(image_path, new)
#

(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3

In [5]:
image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
print(image_path)
# image to array
print(image)
image_array = np.array(image)
resolution = 3.3446  # pixels per micrometer
# get the image size
image_size = image_array.shape
print(image_size)
if image_array.shape[1] <= 500:
    scale_bar_length = 5  # um
    scale_bar_height = 1  # pixels
    padding = 3  # pixels
elif image_array.shape[1] > 500:
    scale_bar_length = 100
    scale_bar_height = 10
    padding = 10
# get the bottom right most corner based on scale 1 % of pixels
scale_bar_x = image_size[1] - (scale_bar_length * resolution) - padding - padding
scale_bar_y = image_size[0] - (scale_bar_height) - padding
print(scale_bar_x, scale_bar_y)
# draw the scale bar
new = cv2.rectangle(
    image_array,
    (int(scale_bar_x), int(scale_bar_y)),
    (image_size[1] - padding, image_size[0] - padding),
    (255, 255, 255),
    -1,
)
# show the image
# plt.imshow(new)
# plt.show()

/home/lippincm/Documents/Interstellar_Analysis/figures/3.figure3/figures/images/PBMC/pyroptosis__apoptosis_pyroptosis__Cells_RadialDistribution_ZernikeMagnitude_CorrPM_6_4_magenta_channel_crop_cell_5.png
[[[  0   0  71]
  [  0   0  62]
  [  0   0  66]
  ...
  [  0   0   5]
  [  0   0   6]
  [  0   0   8]]

 [[  0   0  71]
  [  0   0  58]
  [  0   0  62]
  ...
  [  0   0   8]
  [  0   0   9]
  [  0   0   7]]

 [[  0   0  63]
  [  0   0  60]
  [  0   0  51]
  ...
  [  0   0   6]
  [  0   0   5]
  [  0   0  12]]

 ...

 [[  0   0  32]
  [  0   0  39]
  [  0   0  31]
  ...
  [255 255 255]
  [  0   0  90]
  [  0   0  89]]

 [[  0   0  35]
  [  0   0  35]
  [  0   0  35]
  ...
  [  0   0  92]
  [  0   0  92]
  [  0   0  90]]

 [[  0   0  34]
  [  0   0  33]
  [  0   0  32]
  ...
  [  0   0  90]
  [  0   0 100]
  [  0   0  94]]]
(200, 200, 3)
177.277 196
